

1.   팀번호 : 2
2.   주제: 리그오브레전드 승패 예측
3.   주제선정이유 : 스크래핑 과제를 리그오브레전드 전적 검색으로 했기 때문에 러신머닝과제도 전 과제를 이용하면 좋을 것 같다고 생각함
4. 수집한 자료에 대한 설명 : op.gg 사이트에서 최근 20개 게임의 kda와 시간, 킬관여율과 승패여부를 먼저 수집한다. 수집한 자료를 통해서 다음 게임의 kda, 시간, 킬관여율을 입력하면 승패 여부를 예측해준다



In [1]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 5.1MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 

In [3]:
from selenium import webdriver
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time

In [4]:
url = "https://www.op.gg/summoner/userName="
summoner = input("소환사명을 입력하세요 : ")
base_url = url + quote_plus(summoner)
print(base_url)

소환사명을 입력하세요 : 아리원챔할꺼야
https://www.op.gg/summoner/userName=%EC%95%84%EB%A6%AC%EC%9B%90%EC%B1%94%ED%95%A0%EA%BA%BC%EC%95%BC


In [5]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--start-maximized")
driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)
driver.get(base_url)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [6]:
driver.get_screenshot_as_file('/content/screenshot.png')

True

In [7]:
gameLists = driver.find_elements_by_class_name('GameItemWrap')
print("len: ", len(gameLists))
print("type: ", type(gameLists))

len:  20
type:  <class 'list'>


In [14]:
games = []

for game in gameLists:
  gameLength = game.find_element_by_class_name("GameLength")
  gameLength = int(gameLength.text.split('m')[0]) * 60 + int(gameLength.text.split('m')[1].split('s')[0])
  # ["23", " 50s"]
  kill = game.find_element_by_class_name("Kill")
  kill = int(kill.text)

  death = game.find_element_by_class_name("Death")
  death = int(death.text)

  assist = game.find_element_by_class_name("Assist")
  assist = int(assist.text)

  killRate = game.find_element_by_class_name("CKRate")
  killRate = int(killRate.text.split()[1].split('%')[0])

  games.append([gameLength, kill, death, assist, killRate])
  print(gameLength, kill, death, assist, killRate)

print(games)

1306 3 3 5 25
996 4 2 2 35
2101 3 6 4 16
924 4 4 0 80
1241 1 0 7 47
978 0 1 0 0
1234 7 3 8 58
1320 6 3 1 44
1400 2 3 0 8
1525 5 5 1 19
2100 9 4 11 67
1065 7 0 1 47
2377 14 3 5 44
1804 4 3 4 47
2057 9 8 9 37
1923 8 7 3 37
2857 9 7 2 33
2599 7 16 19 36
2271 10 8 6 35
1605 6 3 3 24
[[1306, 3, 3, 5, 25], [996, 4, 2, 2, 35], [2101, 3, 6, 4, 16], [924, 4, 4, 0, 80], [1241, 1, 0, 7, 47], [978, 0, 1, 0, 0], [1234, 7, 3, 8, 58], [1320, 6, 3, 1, 44], [1400, 2, 3, 0, 8], [1525, 5, 5, 1, 19], [2100, 9, 4, 11, 67], [1065, 7, 0, 1, 47], [2377, 14, 3, 5, 44], [1804, 4, 3, 4, 47], [2057, 9, 8, 9, 37], [1923, 8, 7, 3, 37], [2857, 9, 7, 2, 33], [2599, 7, 16, 19, 36], [2271, 10, 8, 6, 35], [1605, 6, 3, 3, 24]]


In [17]:
results = []

for game in gameLists:
  GameResult = game.find_element_by_class_name("GameResult")
  if GameResult.text == 'Victory':
    results.append(1)
  else:
    results.append(0)
print(results)

[1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1]


In [19]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [21]:
X = np.array(games)
Y = np.array(results)

reg = LinearRegression().fit(X, Y)
reg.score(X, Y)

0.35356298059977465

In [27]:
pred = [[21* 60 + 46, 3, 3, 5, 25]]
pred = np.array(pred)
pred

array([[1306,    3,    3,    5,   25]])

In [28]:
predRes = reg.predict(pred)
predRes

array([0.64812879])

In [30]:
import math
math.floor(predRes)

0